In [2]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

SAMPLE_FP = Path("sample.jsonl")

def read_jsonl(fp: Path) -> pd.DataFrame:
    rows = []
    with fp.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return pd.DataFrame(rows)

df = read_jsonl(SAMPLE_FP)

# --- Flatten dict columns ---
state = pd.json_normalize(df["state"]).add_prefix("st.")
pheno = pd.json_normalize(df["phenotype"]).add_prefix("ph.")
local = pd.json_normalize(df["local"]).add_prefix("loc.")
pos   = pd.json_normalize(df["pos"]).add_prefix("pos.")

g = pd.concat(
    [df.drop(columns=[c for c in ["state","phenotype","local","pos"] if c in df.columns]),
     state, pheno, local, pos],
    axis=1
)

# Ensure numerics
for c in ["t","age","agent_id","pop_n", "st.E","st.M","st.D","ph.A_mature","ph.E_repro_min","ph.M_repro_min","st.repro_cd_s"]:
    if c in g.columns:
        g[c] = pd.to_numeric(g[c], errors="coerce")

print("Flattened columns:", [c for c in g.columns if c.startswith(("st.","ph.","loc.","pos."))][:20], " ...")
display(g.head(3))

# --- Gates (match your new rule) ---
# energy gate: Et >= E_repro_min * M   (as you proposed)
Et   = g["st.E"]
M    = g["st.M"]
Amat = g["ph.A_mature"]
Emin = g["ph.E_repro_min"]
Mmin = g["ph.M_repro_min"]

g["ok_age"] = g["age"] >= Amat

# cooldown optional: if you don't log it, assume ok
if "st.repro_cd_s" in g.columns:
    g["ok_cd"] = g["st.repro_cd_s"].fillna(0.0) <= 0.0
else:
    g["ok_cd"] = True

g["E_gate_rhs"]    = Emin * M
g["energy_margin"] = Et - g["E_gate_rhs"]
g["ok_E"]          = g["energy_margin"] >= 0.0

g["mass_margin"] = M - Mmin
g["ok_M"]        = g["mass_margin"] >= 0.0

g["ok_ready"] = g["ok_age"] & g["ok_cd"] & g["ok_E"] & g["ok_M"]

def share(s): 
    s = s.dropna()
    return float(s.mean()) if len(s) else float("nan")

print("\nShares:")
print(pd.Series({
    "ok_age": share(g["ok_age"]),
    "ok_cd": share(g["ok_cd"]),
    "ok_E": share(g["ok_E"]),
    "ok_M": share(g["ok_M"]),
    "ok_ready": share(g["ok_ready"]),
}).round(4))

print("\nKey describes:")
display(g[["st.E","st.M","ph.A_mature","ph.E_repro_min","ph.M_repro_min","energy_margin","mass_margin"]].describe().round(6))

# Which gate fails first?
def first_fail(r):
    if not r["ok_age"]: return "age"
    if not r["ok_cd"]:  return "cooldown"
    if not r["ok_E"]:   return "energy"
    if not r["ok_M"]:   return "mass"
    return "ready"

g["fail_reason"] = g.apply(first_fail, axis=1)
display(g["fail_reason"].value_counts().to_frame("count"))

# Per-agent readiness rate (useful)
per_agent = g.groupby("agent_id").agg(
    n=("ok_ready","size"),
    share_ready=("ok_ready","mean"),
    share_okE=("ok_E","mean"),
    share_okM=("ok_M","mean"),
    M_med=("st.M","median"),
    Mreq_med=("ph.M_repro_min","median"),
    Et_med=("st.E","median"),
    Emin_med=("ph.E_repro_min","median"),
    Amat=("ph.A_mature","median"),
).sort_values(["share_ready","share_okM","share_okE"], ascending=False)
display(per_agent.round(4))

Flattened columns: ['st.E', 'st.E_fast', 'st.E_slow', 'st.M', 'st.Fg', 'st.D', 'st.hunger', 'st.speed', 'st.repro_cd_s', 'ph.A_mature', 'ph.repro_rate', 'ph.E_repro_min', 'ph.repro_cost', 'ph.M_repro_min', 'ph.E_rep_min', 'ph.metabolism_scale', 'ph.susceptibility', 'ph.stress_per_drain', 'ph.repair_capacity', 'ph.frailty_gain']  ...


,event,t,agent_id,age,birth_t,traits,pop_n,st.E,st.E_fast,st.E_slow,...,ph.sociability,ph.mobility,ph.cold_aversion,ph.sense_strength,loc.B0,loc.F0,loc.C0,pos.x,pos.y,pos.heading
0,sample,0.0,3,0.02,0.0,"[-0.6437026262283325, 0.751379132270813, 0.353...",12,0.192483,0.150656,0.255225,...,0.478837,0.708768,0.453721,0.727475,1.000000,0.012893,0.023042,35.873061,31.556621,-2.651120
1,sample,1.0,6,1.02,0.0,"[0.11625058203935623, 0.8801883459091187, -0.5...",12,0.132878,0.132563,0.133349,...,0.341044,0.286676,0.455312,0.392084,0.440970,0.046054,0.026810,30.973425,24.094016,2.877148
2,sample,2.0,4,2.02,0.0,"[0.4509298801422119, 0.7387500405311584, -0.07...",12,0.144130,0.166356,0.110790,...,0.574710,0.620196,0.549161,0.508236,0.806122,0.000000,0.023334,30.980999,33.627615,-0.898854



Shares:
ok_age      0.9722
ok_cd       0.9824
ok_E        0.9426
ok_M        0.9084
ok_ready    0.8696
dtype: float64

Key describes:


,st.E,st.M,ph.A_mature,ph.E_repro_min,ph.M_repro_min,energy_margin,mass_margin
count,1081.000000,1081.000000,1081.000000,1081.000000,1081.000000,1081.000000,1081.000000
mean,0.226486,0.410082,23.507057,0.179926,0.332976,0.152828,0.077106
std,0.061346,0.060615,4.834759,0.028413,0.023219,0.056796,0.055586
min,0.000000,0.103275,14.916072,0.154229,0.257027,-0.100906,-0.218245
25%,0.234100,0.394897,19.326166,0.154229,0.321519,0.167001,0.076113
50%,0.244358,0.431463,20.564687,0.168658,0.321519,0.169484,0.088626
75%,0.248376,0.446484,29.879849,0.223243,0.357858,0.179481,0.090636
max,0.338961,0.565381,29.879849,0.232420,0.374179,0.251438,0.229173


,count
fail_reason,
ready,940
energy,61
mass,43
age,30
cooldown,7


,n,share_ready,share_okE,share_okM,M_med,Mreq_med,Et_med,Emin_med,Amat
agent_id,,,,,,,,,
7,349,1.0000,1.0000,1.0000,0.4465,0.3579,0.2447,0.1687,20.5647
19,162,0.9691,1.0000,1.0000,0.4074,0.3215,0.2442,0.1542,29.8798
10,131,0.9618,0.9847,0.9771,0.4695,0.3362,0.2743,0.2234,19.3262
8,210,0.9095,0.9524,0.9143,0.4049,0.3215,0.2423,0.1542,29.8798
1,5,0.8000,0.8000,1.0000,0.3808,0.2959,0.1272,0.1948,28.7717
12,28,0.7500,0.8214,1.0000,0.3937,0.2584,0.2345,0.2030,22.6788
18,28,0.7143,0.7857,1.0000,0.4695,0.3362,0.2686,0.2234,19.3262
9,9,0.6667,0.6667,1.0000,0.4460,0.2878,0.2392,0.2324,21.6868
2,10,0.6000,0.6000,0.7000,0.3091,0.3055,0.1597,0.1801,14.9161


In [3]:
dt = 0.02  # din AP.dt
ready = g[g["ok_ready"]]

print(ready["ph.repro_rate"].describe())
p_step = 1.0 - np.exp(-ready["ph.repro_rate"] * dt)
print(p_step.describe())

# Grov "förväntad antal triggers" i samplet (inte exakt, men ska INTE vara ~0)
print("E[triggers] approx over rows:", float(p_step.sum()))

count    940.000000
mean       0.392037
std        0.016816
min        0.281781
25%        0.378224
50%        0.386005
75%        0.408398
max        0.409823
Name: ph.repro_rate, dtype: float64
count    940.000000
mean       0.007810
std        0.000334
min        0.005620
25%        0.007536
50%        0.007690
75%        0.008135
max        0.008163
Name: ph.repro_rate, dtype: float64
E[triggers] approx over rows: 7.341416696644644


In [4]:
import pandas as pd

life = pd.read_json("life.jsonl", lines=True)

life["event"].value_counts()


event
birth    19
death    17
Name: count, dtype: int64

In [5]:
life[life["event"] == "birth"][["t", "agent_id"]].sort_values("t")

,t,agent_id
0,0.00,1
11,0.00,12
10,0.00,11
8,0.00,9
7,0.00,8
6,0.00,7
9,0.00,10
4,0.00,5
3,0.00,4
2,0.00,3
